In [1]:
!pip install -q datasets
!pip install rouge

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from datasets import load_dataset
import json
from typing import List, Dict, Any
import random
from collections import Counter

class PromptTester:
    def __init__(self, model_name: str, hf_token: str):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            token=hf_token,
            device_map="auto",
            torch_dtype=torch.bfloat16
        )
        self.dataset = load_dataset("n3Er/FinQA-Infix")
        self.examples = self.get_curated_examples()

    def categorize_operation(self, operation: str) -> str:
        """Categorize the type of operation"""
        if any(op in operation for op in ["table_max", "table_min", "table_sum", "table_average"]):
            return "table"
        elif any(op in operation for op in ["greater", "less", "equal"]):
            return "boolean"
        elif "exp" in operation:
            return "exponential"
        else:
            return "arithmetic"

    def get_curated_examples(self) -> List[Dict]:
        """Get diverse examples covering all operation types and operators"""
        train_data = list(self.dataset['train'])
        # Filter for examples with valid operations and answers
        valid_examples = [
            ex for ex in train_data
            if ex['program_re'] and ex['answer'] is not None
        ]
        # Define operation subtypes we want to include
        operation_subtypes = {
            "arithmetic": {
                "addition": "add(",
                "subtraction": "subtract(",
                "multiplication": "multiply(",
                "division": "divide(",
                "percentage": "multiply( divide("
            },
            "table": {
                "sum": "table_sum(",
                "max": "table_max(",
                "min": "table_min(",
                "average": "table_average("
            },
            "boolean": {
                "greater": "greater(",
                "less": "less(",
                "equal": "equal("
            },
            "exponential": {
                "power": "exp("
            }
        }
        # Initialize collection of examples
        selected_examples = []
        # For each main operation type
        for op_type, subtypes in operation_subtypes.items():
            for subtype, operator in subtypes.items():
                # Find examples containing this operator
                matching_examples = [
                    ex for ex in valid_examples
                    if operator in ex['program_re']
                    and not any(operator in selected['program_re'] for selected in selected_examples)
                ]
                # Select the first matching example
                for example in matching_examples:
                    if len(selected_examples) < 8:  # Limit to 8 total examples
                        selected_examples.append(example)
                        break
        return selected_examples

    def format_table(self, table: List[List[Any]]) -> str:
        if not table:
            return ""
        return "\n".join(" | ".join(str(cell) for cell in row) for row in table)

    def format_context(self, example: Dict) -> str:
        """Format context consistently"""
        parts = []
        if example['pre_text']:
            parts.append("Context before:")
            parts.append(' '.join(example['pre_text']))
        if example['table']:
            parts.append("\nTable:")
            parts.append(self.format_table(example['table']))
        if example['post_text']:
            parts.append("\nContext after:")
            parts.append(' '.join(example['post_text']))
        return "\n".join(parts)

    def create_prompt(self, example: Dict) -> str:
        """Create prompt with chain of thought reasoning"""
        demonstrations = []
        for demo in self.examples:  # Using all examples
            demo_text = f"""Question: {demo['question']}
Context:
{self.format_context(demo)}
Think through the solution:
1. Looking at the values in the context:
   - What specific numbers are we comparing or calculating with?
   - Is this a percentage change, ratio, or absolute difference?
2. For percentage changes: use divide(subtract(new_value, old_value), old_value) then multiply by 100
   For ratios: use divide(numerator, denominator)
   For differences: use subtract(new_value, old_value)
3. The exact calculation needed is:
Operations: {demo['program_re']}
4. This gives us:
Result: {demo['answer']}
---"""
            demonstrations.append(demo_text)
        context = self.format_context(example)
        return f"""Solve each financial question by thinking through the steps carefully.
Always write the exact mathematical expression using function notation (add(), subtract(), multiply(), divide(), greater(), less(), table_sum(), table_max(), table_average(), exp(), etc.).
Examples:
{chr(10).join(demonstrations)}
For this question:
Question: {example['question']}
Context:
{context}
Think through your solution:
1. Identify the exact numbers needed from the context
2. For percentage changes: use divide(subtract(new_value, old_value), old_value) then multiply by 100
   For ratios: use divide(numerator, denominator)
   For differences: use subtract(new_value, old_value)
3. Write the complete mathematical expression using function notation
You must provide both:
Operations: [write the exact mathematical expression using function notation]
Result: [result]"""

    def extract_response(self, response: str) -> Dict[str, str]:
        """Extract operations and result with better handling of variations"""
        operations = None
        answer = None
        # Clean up the response
        lines = [line.strip() for line in response.split('\n') if line.strip()]
        # Handle variations in format
        for line in lines:
            if "Operations:" in line:
                operations = line.split("Operations:")[-1].strip()
                operations = operations.replace('[write the exact mathematical expression using function notation]', '').strip()
                operations = operations.replace('[operations]', '').strip()
                if operations == 'None' or operations == '---':
                    operations = None
            elif "Result:" in line:
                answer = line.split("Result:")[-1].strip()
                answer = answer.replace('[result]', '').strip()
                if answer == 'None' or answer == '---' or answer == '(as a number)':
                    answer = None
        return {
            "operations": operations,
            "answer": answer
        }

    def evaluate_response(self, expected_ops: str, predicted_ops: str) -> Dict[str, float]:
        """Evaluate the response using ROUGE metrics"""
        try:
            from rouge import Rouge
        except ImportError:
            print("Rouge package not found. Installing rouge...")
            import subprocess
            import sys
            subprocess.check_call([sys.executable, "-m", "pip", "install", "rouge"])
            from rouge import Rouge
        metrics = {}
        # Initialize Rouge for operation comparison
        rouge = Rouge()
        # Evaluate operations using Rouge if both exist
        if expected_ops and predicted_ops:
            try:
                # Calculate Rouge scores
                rouge_scores = rouge.get_scores(predicted_ops, expected_ops)[0]
                metrics['rouge1_f'] = rouge_scores['rouge-1']['f']
                metrics['rouge2_f'] = rouge_scores['rouge-2']['f']
                metrics['rougeL_f'] = rouge_scores['rouge-l']['f']
            except Exception:
                metrics['rouge1_f'] = 0.0
                metrics['rouge2_f'] = 0.0
                metrics['rougeL_f'] = 0.0
        else:
            metrics['rouge1_f'] = 0.0
            metrics['rouge2_f'] = 0.0
            metrics['rougeL_f'] = 0.0
        return metrics

    def test_prompts(self, num_samples: int = 10) -> None:
        """Test with chain of thought prompting and evaluate using ROUGE"""
        test_samples = []
        example_ids = {ex['id'] for ex in self.examples}
        for sample in random.sample(list(self.dataset['train']), num_samples + len(example_ids)):
            if sample['id'] not in example_ids and len(test_samples) < num_samples:
                test_samples.append(sample)
        print("\nExample Operations Used in Prompt:")
        print("=" * 50)
        for ex in self.examples:
            print(f"\nOperation Type: {self.categorize_operation(ex['program_re'])}")
            print(f"Operations: {ex['program_re']}")
        print("\nTesting with chain of thought reasoning:")
        print("=" * 50)
        # Collect metrics across all samples
        all_metrics = []
        for idx, sample in enumerate(test_samples, 1):
            print(f"\nTest Sample {idx}")
            print("-" * 30)
            print(f"Operation Type: {self.categorize_operation(sample['program_re'])}")
            print(f"Question: {sample['question']}")
            print(f"Expected Operations: {sample['program_re']}")
            print(f"Expected Answer: {sample['answer']}")
            # Get response with chain of thought
            prompt = self.create_prompt(sample)
            inputs = self.tokenizer(prompt, return_tensors="pt").to("cuda")
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=200,
                    temperature=0.7,
                    top_p=0.95,
                    do_sample=True,
                    num_return_sequences=1,
                    pad_token_id=self.tokenizer.eos_token_id
                )
            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            extracted = self.extract_response(response)
            print("\nModel Response:")
            print(f"Operations needed: {extracted['operations']}")
            print(f"Answer: {extracted['answer']}")
            # Evaluate the response using ROUGE
            metrics = self.evaluate_response(
                sample['program_re'],
                extracted['operations']
            )
            all_metrics.append(metrics)
            print("\nROUGE Scores:")
            print(f"ROUGE-1 F1: {metrics['rouge1_f']:.3f}")
            print(f"ROUGE-2 F1: {metrics['rouge2_f']:.3f}")
            print(f"ROUGE-L F1: {metrics['rougeL_f']:.3f}")
        # Calculate and display average metrics
        avg_metrics = {
            metric: sum(m[metric] for m in all_metrics) / len(all_metrics)
            for metric in all_metrics[0].keys()
        }
        print("\nOverall ROUGE Scores:")
        print("=" * 50)
        print(f"Average ROUGE-1 F1: {avg_metrics['rouge1_f']:.3f}")
        print(f"Average ROUGE-2 F1: {avg_metrics['rouge2_f']:.3f}")
        print(f"Average ROUGE-L F1: {avg_metrics['rougeL_f']:.3f}")

if __name__ == "__main__":
    tester = PromptTester(
        model_name="Qwen/Qwen2.5-7B-Instruct",
        hf_token="your_private_token"
    )
    tester.test_prompts(num_samples=10)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



Example Operations Used in Prompt:

Operation Type: arithmetic
Operations: add(75.0, 72.7), divide(#0, 20)

Operation Type: arithmetic
Operations: subtract(959.2, 991.1), divide(#0, 991.1)

Operation Type: boolean
Operations: multiply(607, 18.13), multiply(#0, const_1000), multiply(3.3, const_1000000), greater(#1, #2)

Operation Type: table
Operations: table_sum(cash flows on retained interests, none)

Operation Type: table
Operations: table_max(cumulative foreign currency translation, none)

Operation Type: table
Operations: table_min(expected volatility, none), table_max(expected volatility, none), subtract(#1, #0)

Operation Type: table
Operations: table_average(2016, none)

Operation Type: exponential
Operations: add(const_1, 2.0%), exp(#0, 7), multiply(397, #1)

Testing with chain of thought reasoning:

Test Sample 1
------------------------------
Operation Type: arithmetic
Question: what was the purchase price of common stock acquired under the espp in 2018?
Expected Operations: